In [2]:
from llm.chatbot import chat_model
from langchain.schema.messages import HumanMessage, SystemMessage

In [4]:
messages = [
     SystemMessage(
         content="""You're an assistant knowledgeable about
         healthcare. Only answer healthcare-related questions."""
     ),
     HumanMessage(content="What is Medicaid managed care?"),
]
chat_model.invoke(messages).content

"Medicaid managed care is a system where state Medicaid programs contract with managed care organizations (MCOs) to provide healthcare services to Medicaid enrollees. In this model, the MCOs assume the risk and responsibility for providing a defined set of healthcare services to enrolled Medicaid beneficiaries, often in exchange for a fixed monthly payment per member.\n\nUnder Medicaid managed care, enrollees typically choose a primary care provider (PCP) from a network of participating healthcare providers who coordinate their care. The MCOs manage the delivery of healthcare services, including authorizing and paying for medically necessary services, coordinating specialty care, and emphasizing preventive care.\n\nMedicaid managed care plans can vary in their specific designs and may include models such as:\n\n1. Health Maintenance Organizations (HMOs): Enrollees typically need to seek care from providers within the HMO's network, except in emergencies.\n\n2. Preferred Provider Organi

In [11]:
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)

review_system_template_str = """Your job is to use patient
reviews to answer questions about their experience at a
hospital. Use the following context to answer questions.
Be as detailed as possible, but don't make up any information
that's not from the context. If you don't know an answer, say
you don't know.

{context}
"""

review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"], template=review_system_template_str
    )
)

review_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["question"], template="{question}"
    )
)

messages = [review_system_prompt, review_human_prompt]
review_prompt_template = ChatPromptTemplate(
    input_variables=["context", "question"],
    messages=messages,
)
context = "I had a great stay!"
question = "Did anyone have a positive experience?"

review_prompt_template.format_messages(context=context, question=question)

[SystemMessage(content="Your job is to use patient\nreviews to answer questions about their experience at a\nhospital. Use the following context to answer questions.\nBe as detailed as possible, but don't make up any information\nthat's not from the context. If you don't know an answer, say\nyou don't know.\n\nI had a great stay!\n"),
 HumanMessage(content='Did anyone have a positive experience?')]

In [17]:
import os
import pandas as pd
from dotenv import load_dotenv, find_dotenv


load_dotenv(find_dotenv())
COHERE_API_KEY = os.environ["COHERE_API_KEY"]
REVIEWS_CSV_PATH = "data/reviews.csv"
REVIEWS_CHROMA_PATH = "llm/chroma_data"


# loader = CSVLoader(file_path=REVIEWS_CSV_PATH, source_column="review")
# reviews = loader.load()

# reviews_vector_db = Chroma.from_documents(
#     reviews, CohereEmbeddings(), persist_directory=REVIEWS_CHROMA_PATH
# )

## Cut DataFrame 20 rows

In [20]:
import pandas as pd
visits = pd.read_csv("data/visits.csv")
visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9998 entries, 0 to 9997
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   patient_id             9998 non-null   int64  
 1   date_of_admission      9998 non-null   object 
 2   billing_amount         9998 non-null   float64
 3   room_number            9998 non-null   int64  
 4   admission_type         9998 non-null   object 
 5   discharge_date         9498 non-null   object 
 6   test_results           9998 non-null   object 
 7   visit_id               9998 non-null   int64  
 8   physician_id           9998 non-null   int64  
 9   payer_id               9998 non-null   int64  
 10  hospital_id            9998 non-null   int64  
 11  chief_complaint        3000 non-null   object 
 12  treatment_description  3000 non-null   object 
 13  primary_diagnosis      3000 non-null   object 
 14  visit_status           9998 non-null   object 
dtypes: f

In [21]:
visits = visits[:20]
visits.to_csv("data/visits_20.csv", index=False)

In [22]:
patient = pd.read_csv('data/patients.csv')
patient = patient[:20]
patient.to_csv("data/patients_20.csv", index=False)

In [24]:
physician = pd.read_csv("data/physicians.csv")
physician = physician[:20]
physician.to_csv("data/physician_20.csv", index=False)

We will use data visit to build relationship every data using id, something like visit_id, physician_id, payer_id, hospital_id and patient_id and review_id from data reviews.csv

## MultiVector

In [48]:
from langchain.storage import InMemoryByteStore
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import Chroma
from langchain_cohere import CohereEmbeddings
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [49]:
loaders = [
    CSVLoader("data/summary/hospital_summary.csv", source_column="summary"),
    CSVLoader("data/summary/patient_summary.csv", source_column="summary"),
    CSVLoader("data/summary/payer_summary.csv", source_column="summary"),
    CSVLoader("data/summary/physician_summary.csv", source_column="summary"),
    CSVLoader("data/summary/visit_summary.csv", source_column="summary"),
    CSVLoader("data/reviews.csv", source_column="review"),
]

docs = []
for loader in loaders:
    docs.extend(loader.load())
text_splitter = RecursiveCharacterTextSplitter()
docs = text_splitter.split_documents(docs)

In [51]:
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="full_documents", embedding_function=CohereEmbeddings(cohere_api_key=COHERE_API_KEY)
)
# The storage layer for the parent documents
store = InMemoryByteStore()
id_key = "doc_id"
# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)
import uuid

doc_ids = [str(uuid.uuid4()) for _ in docs]

In [56]:
retriever.vectorstore.add_documents(docs)

In [ ]:
# Vectorstore alone retrieves the small chunks
retriever.vectorstore.similarity_search("justice breyer")[0]

## Test Review Chain

In [3]:
from llm_api.src.chains.review_chain import setup_vector_chain
import time
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

query = """give me the best patient review."""
vector_chain = setup_vector_chain()
start_time = time.time()
response = vector_chain.invoke(query)
end_time = time.time()
response_time = end_time - start_time
res = response.get("result")
print(f"\nResponse time: {response_time} seconds")
print("Response :", res)

2024-05-29 19:39:36 [INFO] Setting up vector chain...
2024-05-29 19:39:36 [INFO] Load CohereEmbedding...
2024-05-29 19:39:37 [INFO] Attempting to set up Neo4j vector index from existing index...
2024-05-29 19:39:43 [INFO] HTTP Request: POST https://api.cohere.ai/v1/chat "HTTP/1.1 200 OK"



Response time: 3.231100559234619 seconds
Response : The patient expresses gratitude for the compassionate and thorough medical care received at the hospital. The medical team is praised for their attentive and supportive nature, ensuring a comfortable and smooth recovery. The facilities are described as top-notch, modern, and well-equipped, contributing to a positive overall experience. This review highlights the excellent standard of care and the patient's satisfaction with their stay.


## Test Cypher Chain
#### Llama 3 8b

In [30]:
question = [
    "What is the average billing amount for Medicaid visits?",
    "How many patients did Jason Williams treat while working at the hospital?",
    "I was looking for a patient but forgot his name,  I only know his date of birth is 1989-01-26 and his blood type is AB+",
    "I need information about the visit on 2022-03-08 such as chief complaint, treatment, who was the patient and the attending physician and payment status",
    "How many patients has Dr. Ryan Brown treated?",
    "How much was billed for patient 789's stay?",
    "Which physician has billed the most to cigna?",
]

result = []

In [32]:
from llm_api.src.chains.cyper_chain import create_cypher_qa_chain
import time
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())


cypher_qa_chain = create_cypher_qa_chain()
start_time = time.time()
for query in question:
    try:
        response = cypher_qa_chain.invoke(query)
        res = response.get("result")
        data = {"Question": query, "Response": res}
        result.append(data)
    except Exception as e:
        data = {"Question": query, "Error": str(e)}
        result.append(data)
    
end_time = time.time()
response_time = end_time - start_time
# res = response.get("result")
print(f"\nResponse time: {response_time} seconds")
# print("Response :", res)



> Entering new GraphCypherQAChain chain...


2024-05-31 19:29:00 [INFO] HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Generated Cypher:
MATCH (p:Payer {name: 'Medicaid'})<-[:COVERED_BY]-(v:Visit)-[t:TREATS]-(phy:Physician)
RETURN avg(t.billing_amount) AS average_billing_amount
Full Context:
[{'average_billing_amount': None}]


2024-05-31 19:29:02 [INFO] HTTP Request: POST https://api.cohere.ai/v1/chat "HTTP/1.1 200 OK"



> Finished chain.


> Entering new GraphCypherQAChain chain...


...


> Finished chain.


> Entering new GraphCypherQAChain chain...


2024-05-31 19:29:18 [INFO] HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Generated Cypher:
MATCH (p:Payer)<-[c:COVERED_BY]-(v:Visit)-[t:TREATS]-(phy:Physician)
WHERE p.name = 'Cigna'
RETURN phy.name AS physician_name, SUM(c.billing_amount) AS total_billed
ORDER BY total_billed DESC
LIMIT 1
Full Context:
[{'physician_name': 'Michael Watson', 'total_billed': 395520.74246200576}]


2024-05-31 19:29:19 [INFO] HTTP Request: POST https://api.cohere.ai/v1/chat "HTTP/1.1 200 OK"



> Finished chain.

Response time: 20.672751665115356 seconds


In [34]:
for res in result:
    print(f"Question: {res['Question']}")
    if "Response" in res:
        print(f"Response: {res['Response']}\n")
    else:
        print(f"Error: {res['Error']}\n")

Question: What is the average billing amount for Medicaid visits?
Response: Unfortunately, I don't know the average billing amount for Medicaid visits.

Question: How many patients did Jason Williams treat while working at the hospital?
Response: Jason Williams treated 18 patients during his time at the hospital.

Question: I was looking for a patient but forgot his name,  I only know his date of birth is 1989-01-26 and his blood type is AB+
Response: I can confirm that the patient with the date of birth 26th of January 1989 and blood type AB+ is Mrs. Brandy Flowers.

Question: I need information about the visit on 2022-03-08 such as chief complaint, treatment, who was the patient and the attending physician and payment status
Response: On March 8, 2022, Malik Sullivan visited with Dr. Christine Perkins regarding impaired coordination and tremors. The treatment plan included neurological assessments, medication for tremor control, and a recommendation for physical therapy. Teresa Caldw

Two tries, the llama3 8b model managed to record a time of 20 seconds with a score of 80% (own assumption)

#### Model Llama 3 70B

In [39]:
from llm_api.src.chains.cyper_chain import create_cypher_qa_chain
import time
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())


cypher_qa_chain = create_cypher_qa_chain()
start_time = time.time()
result = []
for query in question:
    try:
        response = cypher_qa_chain.invoke(query)
        res = response.get("result")
        data = {"Question": query, "Response": res}
        result.append(data)
    except Exception as e:
        data = {"Question": query, "Error": str(e)}
        result.append(data)
    
end_time = time.time()
response_time = end_time - start_time
# res = response.get("result")
print(f"\nResponse time: {response_time} seconds")
# print("Response :", res)



> Entering new GraphCypherQAChain chain...


2024-05-31 19:36:47 [INFO] HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Generated Cypher:
MATCH (p:Payer {name: 'Medicaid'})<-[:COVERED_BY]-(v:Visit)-[t:TREATS]-(phy:Physician)
RETURN avg(t.billing_amount) AS average_billing_amount
Full Context:
[{'average_billing_amount': None}]


2024-05-31 19:36:49 [INFO] HTTP Request: POST https://api.cohere.ai/v1/chat "HTTP/1.1 200 OK"



> Finished chain.


> Entering new GraphCypherQAChain chain...


...


> Finished chain.


> Entering new GraphCypherQAChain chain...


2024-05-31 19:37:04 [INFO] HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Generated Cypher:
MATCH (p:Payer)<-[c:COVERED_BY]-(v:Visit)-[t:TREATS]-(phy:Physician)
WHERE p.name = 'Cigna'
RETURN phy.name AS physician_name, SUM(c.billing_amount) AS total_billed
ORDER BY total_billed DESC
LIMIT 1
Full Context:
[{'physician_name': 'Michael Watson', 'total_billed': 395520.74246200576}]


2024-05-31 19:37:05 [INFO] HTTP Request: POST https://api.cohere.ai/v1/chat "HTTP/1.1 200 OK"



> Finished chain.

Response time: 19.84330677986145 seconds


In [40]:
for res in result:
    print(f"Question: {res['Question']}")
    if "Response" in res:
        print(f"Response: {res['Response']}\n")
    else:
        print(f"Error: {res['Error']}\n")

Question: What is the average billing amount for Medicaid visits?
Response: Unfortunately, I do not know the average billing amount for Medicaid visits.

Question: How many patients did Jason Williams treat while working at the hospital?
Response: Jason Williams treated 18 patients during his time at the hospital.

Question: I was looking for a patient but forgot his name,  I only know his date of birth is 1989-01-26 and his blood type is AB+
Response: I can confirm that the patient with the date of birth 1989-01-26 and blood type AB+ is Mrs. Brandy Flowers.

Question: I need information about the visit on 2022-03-08 such as chief complaint, treatment, who was the patient and the attending physician and payment status
Response: On March 8, 2022, Malik Sullivan saw Dr. Christine Perkins with a chief complaint of impaired coordination and tremors. Dr. Perkins recommended neurological assessments, prescribed medications for tremor control, and suggested physical therapy. The payment statu

## Test Rag agent
**Note:** sometimes we need to rerun the code to get the best results

In [17]:
from llm_api.src.agents.rag_agent import agent_executor

agent = agent_executor()

ValueError: Prompt missing required variables: {'agent_scratchpad', 'tools', 'tool_names'}

In [15]:
import time
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

from llm_api.src.agents.rag_agent import agent_executor

# question = {"input": "What is the wait time at Wallace-Hamilton?"}
question = {
         "input": (
             "What have patients said about their "
             "quality of rest during their stay?"
         )
     }

# question = {
#     "input": (
#         "Which state had the largest percent increase "
#         "in Medicaid visits from 2022 to 2023?"
#     )
# }

agent = agent_executor()
start_time = time.time()
# response = agent.invoke(question)
# end_time = time.time()
# response_time = end_time - start_time
# res = response.get("output")
# print(f"\nResponse time: {response_time} seconds")
# print("Response :", res)

ValueError: Prompt missing required variables: {'agent_scratchpad', 'tools', 'tool_names'}

In [7]:
from langchain import hub
prompt = hub.pull("hwchase17/react")

In [8]:
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

## Test Api LLM on local dev

```bash
cd 
```

In [21]:
import time
import requests

CHATBOT_URL = "http://127.0.0.1:8000/rag-agent"

questions = [
   "What is the current wait time at Wallace-Hamilton hospital?",
   "Which hospital has the shortest wait time?",  # <--ini error
   "At which hospitals are patients complaining about billing and insurance issues?",
   "What is the average duration in days for emergency visits?", # <--ini error
   "What are patients saying about the nursing staff at Castaneda-Hardy?",
   "What was the total billing amount charged to each payer for 2023?",
   "What is the average billing amount for Medicaid visits?",  # <--ini error
   "How many patients has Dr. Ryan Brown treated?",
   "Which physician has the lowest average visit duration in days?", # <--ini error
   "How many visits are open and what is their average duration in days?",
   "Have any patients complained about noise?",
   "How much was billed for patient 789's stay?",
   "Which physician has billed the most to cigna?",
   "Query the graph database to show which states had the largest percentage increase in Medicaid visits from 2022 to 2023?"
]

request_bodies = [{"query": q} for q in questions]

def send_requests(request_bodies):
    responses = []
    for data in request_bodies:
        try:
            response = requests.post(CHATBOT_URL, json=data)
            response.raise_for_status()
            responses.append(response.json())
        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}")
            responses.append(None)
    return responses

start_time = time.perf_counter()
outputs = send_requests(request_bodies)
end_time = time.perf_counter()

print(f"Run time: {end_time - start_time} seconds")

Request failed: 500 Server Error: Internal Server Error for url: http://127.0.0.1:8000/rag-agent
Run time: 885.4596790479991 seconds


In [27]:
885.4596790479991 / 60

14.75766131746665

In [22]:
outputs

[{'input': 'What is the current wait time at Wallace-Hamilton hospital?',
  'output': 'The current wait time at Wallace-Hamilton hospital is 5 hours 34 minutes.',
  'intermediate_steps': ["(AgentAction(tool='Waits', tool_input='Wallace-Hamilton', log='Thought: I need to find the current wait time at Wallace-Hamilton hospital.\\n\\nAction: Waits\\nAction Input: Wallace-Hamilton'), '5 hours 34 minutes')"]},
 {'input': 'Which hospital has the shortest wait time?',
  'output': 'Lewis-Nelson hospital has the shortest wait time of 8 minutes.',
  'intermediate_steps': ["(AgentAction(tool='Availability', tool_input='(no input needed)', log='Thought: I need to find out which hospital has the shortest wait time.\\n\\nAction: Availability\\nAction Input: (no input needed)'), {'lewis-nelson': 8})"]},
 {'input': 'At which hospitals are patients complaining about billing and insurance issues?',
  'output': 'Agent stopped due to iteration limit or time limit.',
  'intermediate_steps': ['(AgentAction(

In [26]:
import asyncio
import time
import httpx


async def make_async_post(url, data):
    timeout = httpx.Timeout(timeout=120)
    async with httpx.AsyncClient() as client:
        response = await client.post(url, json=data, timeout=timeout)
        return response

async def make_bulk_requests(url, data):
    tasks = [make_async_post(url, payload) for payload in data]
    responses = await asyncio.gather(*tasks)
    outputs = [r.json()["output"] for r in responses]
    return outputs

def run_asyncio_task(task):
    if asyncio.get_event_loop().is_running():
        new_loop = asyncio.new_event_loop()
        asyncio.set_event_loop(new_loop)
        result = new_loop.run_until_complete(task)
        asyncio.set_event_loop(asyncio.get_event_loop())
    else:
        result = asyncio.run(task)
    return result

request_bodies = [{"query": q} for q in questions]

start_time = time.perf_counter()
outputs = run_asyncio_task(make_bulk_requests(CHATBOT_URL, request_bodies))
end_time = time.perf_counter()

print(f"Run time: {end_time - start_time} seconds")

RuntimeError: Cannot run the event loop while another loop is running